In [ ]:
#This notebook contains an implementation to solve the problem min Sum ||A-B_i||_1 + Lambda||2A-11^t||_nucl.
#We solve the problem an the implementation based on "The Augmented Lagrange Multiplier
#method for exact recovery of corrupted low-rank matrices" and some notes of Boyd for ADMM.
#which we link here. https://web.stanford.edu/~boyd/papers/pdf/admm_slides.pdf

#"/although interior point methods normally take very few iterations t converge, they have difficulty in
#handling large matrices because of the complexity of computing the step direction is O(m^6), where m is the dimension of 
#matrix [...] generic interior point solvers are too limited for Robust PCA ".

#Note that the robust PCA is used in many contexts. Ours is the clique problem which deals with nxn symetric matrices
#. However this implementation works for m x n matrices. Our implementation is an extension of the robust pca algorithm.


In [1]:
#admmRecovery does not call mosek. It uses the prox operation to find xi+1 and Z.
# See the paper "Graph clustering and the nuclear Wasserstein metric." for the explanation of this algorithm
# ulr: - soon to appear in ArXiv - 
# observations : a vector containing the observed adjacency matrices.
# lambda: the regularization parameter to be used.
#n : number of vertices of the graphs.
# rho: parameter for admm algorithm.
#stopCrit1: stoping criterium for the algorithm.
#maxIter: Maximum number of iterations of the ADMM algorithm.
function admmRecovery(observations,lambda,n,rho=1.6,stopCrit1=1.0e-5,maxIter=200)
    
    #Important! all matrices in the observations are assumed to have 1 in the diagonal.
    #Transformation of the B_i to the C_i where C_i = 2B_i-11^t
    observationsTransformed = transform(observations,n)
    #Initialization (All parameters are based on the paper.)
    k = 0
    N = length(observationsTransformed)
    averObservations=AvergeMatrix(observationsTransformed)
    Y = Array{Any}(N)
    [Y[r]=observationsTransformed[r]/J(observationsTransformed[r],lambda) for r in 1:N]
    Z= averObservations
    softZ = zeros(Z)
    Zviejo= zeros(Z)
    Xviejo = Array{Any}(N)
    [Xviejo[r]=zeros(Z) for r in 1:N]
    
    Xnuevo = Array{Any}(N)
    [Xnuevo[r]=zeros(Z) for r in 1:N]
    
    #mu = 1.25/norm(averObservations,2)
    #Iterations
    #In order to avoid evaluation of stoping criteria in the first loop, we do a while true and then 
    #add breaks if the stoping criteria are met.
    while true 
                
        # First solve x^i_k+1 = arg min 1/2||x^i_k-C_i||+mu/2||x^i_k-Z_k+y^i_k||_2^2
      #  println("solving for x...")
        [Xnuevo[i] = solveForXprox(observationsTransformed[i],Y[i],Z,rho,n) for i in 1:N]
       

       # println("Solved, now solving for Z")
       # tic()
         # now we solve Z_k+1 = arg min prox_g,rho (1/N sum(x_i_k+1 + 1/rho y_i^k))
        singValDesc = svdfact!(AvergeMatrix(Xnuevo +(1/rho)*Y))
        #SVD(A) returns a triple (U,S,V) where S contains the singular values and A=U*S*V' (' denotes ')
        #for SVD(A) to work correctly, A must be m x n where m>= n or else S wont have the proper dimensions.
        softZ = diagm(SoftThreshold.(singValDesc[:S],lambda/rho))
        softZ = convert(Array{Float64,2},softZ)    
        Z = singValDesc[:U]*softZ*singValDesc[:Vt]       
       # toc()
       # println("solved. now solving Y")
        #update Y
        [Y[i] = Y[i]+ rho*(Xnuevo[i]-Z)for i in 1:N]
        
        
        #compute the residuals to check stopping conditions and compute rho
        residual = AvergeMatrix(Xnuevo)-Z
        residualDual = rho*(Zviejo-Z)
        #check stopping condition
     #   println("Cheking stop criteriums and updating.")
        if  firstCriterium(stopCrit1,n,residual,residualDual)
     #       println("Done")
           break
        end
        #update rho
        rho = updaterho(rho,residual,residualDual)
         
        #update Xviejo
        Xviejo = Xnuevo
        #update Zviejo
        Zviejo = Z
        # update k 
    #    println("Step $(k)")
    #    println(norm(Xnuevo[1],2))
        k = k+1
        
     #   println(rho)
        if k>maxIter
            println("exceeded max number of iteration at solving")
            return((Z,Xnuevo))
            break
        end    
    end
    return((Z,Xnuevo))
end
        
        
        

admmRecovery (generic function with 4 methods)

In [2]:
function solveForXprox(obs,Y,Z,mu,n)
    actualizado = SoftThreshold(Z-(1/mu)*Y-obs,1/(2*mu))+obs
    return(actualizado)
end    


solveForXprox (generic function with 1 method)

In [3]:
#Aux functions


#perturbation operator. Perturbs every entry of the matrix W using the function f.
#W is a m x n matriz to perturb
#epsilon is the perturbation
function SoftThreshold(W,perturbation)
    map(W) do x
        perturb(x,perturbation)
    end
end



#Method for perturbing x given epsilon. Used in the softThresholding function.
function perturb(x,epsilon)
    if x>epsilon 
        return x-epsilon
    end
    if x<-epsilon
        return x+epsilon
    else
        return(0)
    end
end



#Method to average matrices

function AvergeMatrix(observations)
    size = length(observations)
    return (1/size)*sum(observations)
end

#J operator
#W is an m x n matriz
#lambda : parameter of the algorithm
function J(W,lambda)
    return max(norm(W,2),(1/lambda)*maximum(abs.(W)))
end

#Function  transform the observed matrices to the desired form. All observed matrices
# have 1 on the diagonal

function transformMatrix(mat,n)
    return(2*mat-ones(n)*ones(n)') 
end

function transform(observations,n)
    return(transformMatrix.(observations,n))
end



function firstCriterium(stopCrit1,n,residual,residualDual)
    
    if  sqrt(n)*norm(residual,2)<= stopCrit1 && sqrt(n)*norm(residualDual,2)<= stopCrit1
        return(true)
    end
    return false
end

# Updating functions

function updaterho(rho,residual,residualDual)
    rhoNew = rho
    if norm(residual,2)>=10*norm(residualDual,2)
    rhoNew = 2*rho
    end
    if norm(residualDual,2)>=10*norm(residual,2)
        rhoNew = rho/2
    end
    return(rhoNew)
end
        


updaterho (generic function with 1 method)

#Another verison of the algorithm wich does use an interior point solver to solve the update steps. Deprectated in favor #admmRecovery.
#admmRecovery2 does calls mosek to find xi+1 and Z.


function admmRecovery2(observations,lambda,n,rho=1.6,stopCrit1=1.0e-7,stopCrit2=1.0e-5,maxIter=10000)
    
    #Important! all matrices in the observations are assumed to have 1 in the diagonal.
    #Transformation of the B_i to the C_i where C_i = 2B_i-11^t
    observationsTransformed = transform(observations,n)
    #Initialization (All parameters are based on the paper.)
    k = 0
    N = length(observationsTransformed)
    averObservations=AvergeMatrix(observationsTransformed)
    Y = Array{Any}(N)
    [Y[r]=observationsTransformed[r]/J(observationsTransformed[r],lambda) for r in 1:N]
    Z= averObservations
    Xviejo = Array{Any}(N)
    [Xviejo[r]=zeros(Z) for r in 1:N]
    
    Xnuevo = Array{Any}(N)
    [Xnuevo[r]=zeros(Z) for r in 1:N]
    
    #mu = 1.25/norm(averObservations,2)
    mu=0.1
    #Iterations
    #In order to avoid evaluation of stoping criteria in the first loop, we do a while true and then 
    #add breaks if the stoping criteria are met.
    while true 
        
       
        
        # First solve x^i_k+1 = arg min ||x^i_k-C_i||+ <yi^k,xi-z^k>   +1/2||x^i_k-Z_k||_2^2
        #for now, we call mosek to solve this
            [Xnuevo[i] = solveForXMosek(observationsTransformed[i],Y[i],Z,mu,n) for i in 1:N]

            

         # now we solve Z_k+1 = arg min prox_g,rho (1/N sum(x_i_k+1 + 1/rho y_i^k))
        singValDesc = svd(AvergeMatrix(Xnuevo) +(1/mu)*AvergeMatrix(Y))
        #SVD(A) returns a triple (U,S,V) where S contains the singular values and A=U*S*V' (' denotes ')
        #for SVD(A) to work correctly, A must be m x n where m>= n or else S wont have the proper dimensions.
        Z = singValDesc[1]*SoftThreshold(diagm(singValDesc[2]),lambda/(mu*N))*singValDesc[3]'

        #update Y
        [Y[i] = Y[i]+ mu*(Xnuevo[i]-Z)for i in 1:N]
        
        #check stopping condition
      #  if  firstCriterium(stopCrit1,Z,averObservations,Xnuevo)
       #     println("Done")
        #    break
       # end
        #update mu
        #mu = updateMu(mu,rho,Xviejo,Xnuevo,averObservations,stopCrit2)
        #mu = mu*rho
        #update Xviejo
        Xviejo = Xnuevo
        # update k 
        println("Step $(k)")
        println(norm(Xnuevo[1],2))
        k = k+1
        
        println(mu)
        if k>maxIter
            println("exceeded max number of iteration at solving")
            break
        end    
    end
    return((Z,Xnuevo))
end
        

#observations are the observations to compute the function f_i.
#Y is the dual in step k.
#X are the old values of x.
#Z is the objective variable in step k
#mu is the step size.
#using JuMP 
#using Mosek

function solveForXMosek(obs,Y,Z,mu,n)
    m = Model(solver=MosekSolver())
    @variable(m,x[1:n,1:n])
    @variable(m,s[1:n,1:n])
    @constraint(m,-s.<=x-obs)    
    @constraint(m,x-obs.<=s)
    @objective(m,Min,0.5*trace(s'*(ones(n)*ones(n)'))+trace(Y'*(x-Z))+(mu/2)*(trace((x-Z)'*(x-Z))))
     status = solve(m)
    return(getvalue(x))
end
   

